<img align="right" style="max-width: 200px; height: auto" src="cfds_logo.png">

###  Lab 06 - "Supervised Machine Learning - k Nearest Neighbor Classification"

Chartered Financial Data Scientist (CFDS), Autumn Term 2020

In the last lab, you got your hands dirty with supervised learning by using the Gaussian Naive-Bayes (GNB) classifier. You learned how to train a model and to evaluate and interpret its results. In this lab, we will look at another popular algorithm, namely the **k Nearest-Neighbors (kNN)** classifier.

The *discriminative* **k Nearest-Neighbors (kNN)** classifier is a simple, easy to understand, versatile, but powerful machine learning algorithm. Until recently (prior to the advent of deep learning approaches) it was used in a variety of applications such as finance, healthcare, political science, handwriting detection, image recognition and video recognition, e.g. in credit ratings, financial institutes used kNN to predict the solvency of customers.

This classification technique is part of the **discriminative** type of classifiers, which can be distinguished from the **generative** type as shown in the following illustration:

<img align="center" style="max-width: 600px; height: auto" src="supervisedlearning.png">

(Inspired by: 'Machine Learning - A Probabilistic Perspective', Kevin P. Murphy)

As always, pls. don't hesitate to ask all your questions either during the lab, post them in our NextThought lab discussion forum (https://financial-data-science.nextthought.io), or send us an email (using our fds.ai email addresses).

### Lab Objectives:

After today's lab you should be able to:

> 1. Know how to setup a **notebook or "pipeline"** that solves a simple supervised classification task.
> 2. Recognize the distinct **data elements** (features and labels) needed to train and evaluate a supervised machine learning classifier. 
> 3. Understand how a Gaussian **k Nearest-Neighbor (kNN)** classifier can be trained and evaluated.
> 4. Know how to use Python's sklearn library to **train** and **evaluate** arbitrary classifiers.
> 5. Understand how to **evaluate** and **interpret** the classification results.

Before we start let's watch a motivational video:

In [ ]:
from IPython.display import YouTubeVideo
# Microsoft: "AI for Health Program"
# YouTubeVideo('ii-FfE-7C-k', width=800, height=600)

### Setup of the Analysis Environment

Similar to the previous labs, we need to import a couple of Python libraries that allow for data analysis and data visualization. In this lab will use the `Pandas`, `Numpy`, `Scikit-Learn`, `Matplotlib` and the `Seaborn` library. Let's import the libraries by the execution of the statements below:

In [ ]:
# import the numpy, scipy and pandas data science library
import pandas as pd
import numpy as np

# import sklearn data and data pre-processing libraries
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# import sklearn naive.bayes and k-nearest neighbor classifier library
from sklearn.neighbors import KNeighborsClassifier

# import sklearn classification evaluation library
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

# import matplotlib data visualization library
import matplotlib.pyplot as plt
import seaborn as sns

Enable inline Jupyter notebook plotting:

In [ ]:
%matplotlib inline

Use the 'Seaborn' plotting style in all subsequent visualizations:

In [ ]:
plt.style.use('seaborn')

Set random seed of all our experiments - this insures reproducibility.

In [ ]:
random_seed = 42

## 1. k Nearest-Neighbor (kNN) Classification

Now, let's have closer look into the non-parametric method used for supervised classification tasks, referred to as the the **k Nearest-Neighbors (kNN)** algorithm. As you learned during the in lecture k-NN classification, the output of the classifier is a class membership. 

Prior to running **k Nearest-Neighbor (kNN)** classification let's briefly revisit the distinct steps of the algorithm as discussed in the lecture:
<img align="center" style="max-width: 700px; height: auto" src="knn.png">

(Courtesy: Intro to AI & ML lecture, Prof. Dr. Borth, University of St. Gallen)

An object is classified by a majority vote of its neighbors, with the object being assigned to the class most common among its k nearest neighbors (k is a positive integer, typically small). If k = 1, then the object is simply assigned to the class of that single nearest neighbor.

### 1.1. Dataset Download and Data Assessment

Let's try the k Nearest-Neighbour algorithm using the delicious **Wine Dataset**! It is a classic and straightforward multi-class classification dataset.

<img align="center" style="max-width: 600px; height: auto" src="wine_dataset.jpg">

(Source: https://www.empirewine.com)

The data is the results of a chemical analysis of wines grown in the same region in Italy by three different cultivators (types). The dataset consists in total of **178 wines** as well as their corresponding **13 different measurements** taken for different constituents found in the three types of wine. Please, find below the list of the individual measurements (features):

>- `Alcohol`
>- `Malic acid`
>- `Ash`
>- `Alcalinity of ash`
>- `Magnesium`
>- `Total phenols`
>- `Flavanoids`
>- `Nonflavanoid phenols`
>- `Proanthocyanins`
>- `Color intensity`
>- `Hue`
>- `OD280/OD315 of diluted wines`
>- `CProline`

Further details on the dataset can be obtained from the following puplication: *Forina, M. et al, PARVUS - "An Extendible Package for Data Exploration, Classification and Correlation.", Institute of Pharmaceutical and Food Analysis and Technologies, Via Brigata Salerno, 16147 Genoa, Italy.*

Let's load the dataset and conduct a preliminary data assessment: 

In [ ]:
wine = datasets.load_wine()

Print and inspect feature names of the dataset:

In [ ]:
wine.feature_names

Print and inspect the class names of the dataset:

In [ ]:
wine.target_names

Print and inspect the top 10 feature rows of the dataset:

In [ ]:
pd.DataFrame(wine.data, columns=wine.feature_names).head(10)

Print and inspect the top 10 labels of the dataset:

In [ ]:
pd.DataFrame(wine.target).head(10)

Determine and print the feature dimensionality of the dataset:

In [ ]:
wine.data.shape

Determine and print the label dimensionality of the dataset:

In [ ]:
wine.target.shape

Plot the data distributions of the distinct features:

In [ ]:
# init the plot
plt.figure(figsize=(10, 10))

# prepare the dataset to be plotable using seaborn

# convert to Panda's DataFrame
wine_plot = pd.DataFrame(wine.data, columns=wine.feature_names)

# add class labels to the DataFrame
wine_plot['class'] = wine.target

# plot a pairplot of the distinct feature distributions
sns.pairplot(wine_plot, diag_kind='hist', hue='class');

### 1.2. Dataset Pre-Processing

#### 1.2.1. Feature Re-Scaling

Observing the features values of the **Wine Dataset** we will notice that their respective value ranges vary widely. This results in a major challenge for distance based machine learning algorithms such as the **k Nearest-Neighbor** classifier. The **k Nearest-Neighbour** classifier calculates the distance between two observations using a distance measure such as the **Euclidean** or **Manhattan** distance.

If one of the features exhibits a wide range of values, the calculated distance will be governed by this particular feature. Therefore, the range of all features needs to be **re-scaled** or **normalized** to a value range beween in $[0,1]$ or $[-1,1]$ so that each feature contributes approximately proportionately to the final distance. 

One widley used method of feature re-scaling is referred to as **Min-Max Normalization** and is given by:

$$x'={\frac  {x-{\text{min}}(x)}{{\text{max}}(x)-{\text{min}}(x)}}$$

Let's re-scale the distinct feature values of the **Wine Dataset** using **Min-Max Normalization** using the `MinMaxScaler` class of the `sklearn` library:

In [ ]:
# init the min-max scaler
scaler = MinMaxScaler(feature_range=(0, 1), copy=True)

# min-max normalize the distinct feature values
wine_data_norm = scaler.fit_transform(wine.data)

Print and inspect the top 10 feature rows of the normalized dataset:

In [ ]:
pd.DataFrame(wine_data_norm, columns=wine.feature_names).head(10)

Ok, we can observe that all features values have been re-scaled. Let's also statistically validate this observation and determine if all feature values have been re-scaled to a value range between in $[0,1]$:

In [ ]:
pd.DataFrame(wine_data_norm, columns=wine.feature_names).describe()

Looks great. All feature values are indeed in a range between $[0,1]$. Let's also visualize the re-scaled feature values and inspect their distributions:

In [ ]:
# init the plot
plt.figure(figsize=(10, 10))

# prepare the dataset to be plotable using seaborn

# convert to Panda's DataFrame
wine_plot = pd.DataFrame(wine_data_norm, columns=wine.feature_names)

# add class labels to the DataFrame
wine_plot['class'] = wine.target

# plot a pairplot of the distinct feature distributions
sns.pairplot(wine_plot, diag_kind='hist', hue='class');

Excellent, the characteristics of the distinct feature value distributions remained unchanged.

#### 1.2.2. Extraction of Training- and Evaluation-Dataset

To understand and evaluate the performance of any trained **supervised machine learning** model, it is good practice to divide the dataset into a **training set** (the fraction of data records solely used for training purposes) and a **evaluation set** (the fraction of data records solely used for evaluation purposes). Please note, the **evaluation set** will never be shown to the model as part of the training process. All of this is exactly what we did in the prior **Gaussian Naive-Bayes** lab.

<img align="center" style="max-width: 500px; height: auto" src="trainevaldataset.png">

We set the fraction of testing records to **30%** of the original dataset:

In [ ]:
eval_fraction = 0.3

Randomly split the **Wine Dataset** into training set and evaluation set using sklearn's `train_test_split` function:

In [ ]:
# 70% training and 30% evaluation
X_train, X_eval, y_train, y_eval = train_test_split(wine_data_norm, wine.target, test_size=eval_fraction, random_state=random_seed, stratify=None)

Evaluate the training set dimensionality:

In [ ]:
X_train.shape, y_train.shape

Evaluate the evaluation set dimensionality:

In [ ]:
X_eval.shape, y_eval.shape

### 1.3. k Nearest-Neighbor (kNN) Classification

There is again a Python implementation of the **k Nearest-Neighbor (kNN)** classifer available in the `Scikit-Learn` library (https://scikit-learn.org) which we can use of the shelf. Please note, for each classifier, available in the `Scikit-Learn` library, a designated and detailed documentation is provided. It often also includes a couple of practical examples and use cases. The documentation of the **k Nearest-Neighbor (kNN)** classifer can be obtained from the following url: 

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

#### 1.3.1. Nearest Neighbors Classification, k=4

Set the number of neighbors `k` to be considered in the classification of each sample: 

In [ ]:
k_nearest_neighbors = 4

Please, recall that we discussed two distinct distance measures in the lecture to calculate the distance between an observation $x$ and it's $k$-nearest-neighbors $x'_{j}$ in a $n$-dimensonal feature space:

**Manhattan distance ("L1-norm"):** $$ D(x, x')=\|\sum^k_{j=1}\sum^n_{i=1}(x_{i} - x'_{j,i})\|$$

**Euclidian distance ("L2-norm"):** $$ D(x, x')=\sqrt{\sum^k_{j=1}\sum^n_{i=1}(x_{i} - x'_{j,i})^2}$$

where the index $j$ denotes the number of $k$-nearest-neighbors and the index $i$ denotes the $i$-th feature of a single nearest neighbor $x_j$. Since the 13 features of the Wine dataset consist of continuous features we will use the Euclidean distance as the distance metric in our kNN classification:

In [ ]:
distance_metric = 'euclidean'

Init the **kNN classifier** of Python's `Scikit-Learn` library of data science algorithms: 

In [ ]:
knn = KNeighborsClassifier(n_neighbors=k_nearest_neighbors, metric=distance_metric)

Train the k-NN classifier using the training dataset:

In [ ]:
knn.fit(X_train, y_train);

Utilize the trained model to predict the response for the evaluation dataset:

In [ ]:
y_pred = knn.predict(X_eval)

Let's have a look at the predicted class labels:

In [ ]:
y_pred

As well as the true class labels:

In [ ]:
y_eval

Determine **prediction accuracy** of the trained model on the evaluation dataset:

In [ ]:
print("Accuracy, k=4: ", metrics.accuracy_score(y_eval, y_pred))

Determine and plot the **confusion matrix** of the individual predictions:

In [ ]:
# determine the prediction confusion matrix
mat = confusion_matrix(y_eval, y_pred)

Visualize the **confusion matrix** of the individual predictions determined by the **k=4 Nearest-Neighbor** classifier:

In [ ]:
# init the plot
plt.figure(figsize=(5, 5))

# plot confusion matrix heatmap
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, cmap='YlOrRd_r', xticklabels=wine.target_names, yticklabels=wine.target_names)

# add plot axis labels
plt.xlabel('[true label]', fontsize=14)
plt.ylabel('[predicted label]', fontsize=14)

# add plot title
plt.title('Confusion Matrix - k-NN, k=4', fontsize=14);

Remember that as part of the lecture you learned about several measures to evaluate the quality of a retrieval system, namely **Precision**, **Recall** and **F1-Score**. Let's briefly revisit their definition and subsequently calculate those measures based on the confusion matrix above:

>- The **Precision**, denoted by Precision $=\frac{TP}{TP + FP}$, is the probability that a retrieved document is relevant.
>- The **Recall**, denoted by Recall $=\frac{TP}{TP + FN}$, is the probability that a relevant document is retrieved.
>- The **F1-Score**, denoted by F1-Score $= 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$, combines precision and recall is the harmonic mean of both measures.

In [ ]:
print(classification_report(y_eval, y_pred))

#### 1.3.2. Nearest Neighbors Classification, k=8

Set the number of neighbors `k` to be considered in the classification of each sample: 

In [ ]:
k_nearest_neighbors = 8

Init the **k-NN classifier** of Python's `sklearn` libary of data science algoritms: 

In [ ]:
knn = KNeighborsClassifier(n_neighbors=k_nearest_neighbors, metric=distance_metric)

Train the k-NN classifier using the training dataset:

In [ ]:
knn.fit(X_train, y_train);

Utilize the trained model to predict the response for the evaluation dataset:

In [ ]:
y_pred = knn.predict(X_eval)

Determine **prediction accuracy** of the trained model on the evaluation dataset:

In [ ]:
print("Accuracy, k=8: ", metrics.accuracy_score(y_eval, y_pred))

Determine and plot the **confusion matrix** of the individual predictions:

In [ ]:
# determine the prediction confusion matrix
mat = confusion_matrix(y_eval, y_pred)

Visualize the **confusion matrix** of the individual predictions determined by the **k=8 Nearest-Neighbor** classifier:

In [ ]:
# init the plot
plt.figure(figsize=(5, 5))

# plot confusion matrix heatmap
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, cmap='YlOrRd_r', xticklabels=wine.target_names, yticklabels=wine.target_names)

# add plot axis labels
plt.xlabel('[true label]', fontsize=14)
plt.ylabel('[predicted label]', fontsize=14)

# add plot title
plt.title('Confusion Matrix - k-NN, k=8', fontsize=14);

Determine and plot a comprehensive **classification report** of the individual predictions:

In [ ]:
# collect classification report
report = classification_report(y_eval, y_pred)

# print classification report
print(report)

#### 1.3.3. Finding the optimal k of the kNN classifier

Until now we have investigated the Euclidean distance based kNN classifer for two distinct values of $k=4$ and $k=8$. However the question remains: can we locate a value for k that yields an even higher classification accuracy? 

Let's therefore investigate the classification performance of a wider range of distinct $k$ values and in particular compare the corresponding classification accuracy. We will do so in the following by specifying a range of distinct $k$ values ranging from $k=1, ..., 50$:

In [ ]:
# try k=1 through k=50 to be evaluated
k_range = range(1, 51)

To run the kNN classification experiments for different values of $k$ we will define a python loop. The loop iterates over the range of distinct k's and conducts the model training (using the training data) and evaluation (using the evaluation data). The classification accuracy for each $k$ value will be collected and stored in a designated list of accuracy scores: 

In [ ]:
# init the distinct accuracy scores obtained on the evaluation data
eval_accuracy_scores = []

# iterate over the distinct k values
for k in k_range:
    
    # init the k-NN classifier
    knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    
    # train the k-NN classifer on the training data
    knn.fit(X_train, y_train)
    
    # evaluate the k-NN classifier on the training data
    y_train_pred = knn.predict(X_train)
    
    # evaluate the k-NN classifier on the evaluation data
    y_eval_pred = knn.predict(X_eval)
    
    # determine classification accuracy
    accuracy = metrics.accuracy_score(y_eval, y_eval_pred)
    
    # collect the classification accuracy of the current k on the evaluation data
    eval_accuracy_scores.append(accuracy)

Visualizing the collected classification accuracy scores of the distinct $k$ values:

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# plot the classification accuracy of distinct k's
ax.plot(range(1, len(eval_accuracy_scores)+1), eval_accuracy_scores, color='darkred', marker='o')

# add grid
ax.grid(linestyle='dotted')

# add axis range and legends
ax.set_xlabel("[$k$-Nearest-Neighbors]", fontsize=14)
ax.set_ylabel("[% classification accuracy]", fontsize=14)

# add plot title
ax.set_title('k-NN Classification Accuracy', fontsize=14);

Alright, we can nicely observe that a constant high classification accuracy on the held out evaluation dataset can be achieved for $k=19, 23,$ and $29$. This is probably the $k$ value that you may want to use for inferencing and rolling to model out to production.

### Exercises:

We recommend you to try the following exercises as part of the lab:

**1. Train, evaluate and plot the prediction accuracy of the k=1,...,40 Nearest Neighbor models.**

> Write a Python loop that trains and evaluates the prediction accuracy of all k-Nearest Neighbor parametrizations ranging from k=1,...,40. Collect and print the prediction accuracy of each model respectively and compare the results. Plot the prediction accuracy collected for each model above. The plot should display the distinct values of k at the x-axis and the corresponding model prediction accuracy on the y-axis. What kind of behaviour in terms of prediction accuracy can be observed with increasing k?

In [ ]:
# ***************************************************
# INSERT YOUR CODE HERE
# ***************************************************

**2. Train, evaluate and plot the prediction accuracy of the k=1,...,40 Nearest Neighbor models without re-scaling the individual feature values.**

> Write a Python loop that trains and evaluates the prediction accuracy of all k-Nearest Neighbor parametrizations ranging from k=1,...,40. Collect and print the prediction accuracy of each model respectively and compare the results. Plot the prediction accuracy collected for each model above. The plot should display the distinct values of k at the x-axis and the corresponding model prediction accuracy on the y-axis. What kind of behaviour in terms of prediction accuracy can be observed with increasing k? What do you observe when comparing the results of the non re-scaled with the results obtained for the scaled features?

In [ ]:
# ***************************************************
# INSERT YOUR CODE HERE
# ***************************************************

**3. Train, evaluate and plot the prediction accuracy of the k=1,...,40 Nearest Neighbor models using the "Manhattan" distance.**

> Write a Python loop that trains and evaluates the prediction accuracy of all k-Nearest Neighbor parametrizations ranging from k=1,...,40 using the "Manhattan" instead of the "Euclidian" distance metric. Collect and print the prediction accuracy of each model respectively and compare the results. Plot the prediction accuracy collected for each model above. The plot should display the distinct values of k at the x-axis and the corresponding model prediction accuracy on the y-axis. What kind of behaviour in terms of prediction accuracy can be observed with increasing k? What do you observe when comparing the results obtained for the "Manhattan" distance with the ones obtained for the "Euclidean" distance?

In [ ]:
# ***************************************************
# INSERT YOUR CODE HERE
# ***************************************************

### Lab Summary:

In this lab, a step by step introduction into **k Nearest-Neighbor (kNN)** classification is presented. The code and exercises presented in this lab may serves as a starting point for more complex and tailored programs.

You may want to execute the content of your lab outside of the Jupyter notebook environment, e.g. on a compute node or a server. The cell below converts the lab notebook into a standalone and executable python script. Pls. note that to convert the notebook, you need to install Python's **nbconvert** library and its extensions:

In [ ]:
# installing the nbconvert library
!pip install nbconvert
!pip install jupyter_contrib_nbextensions

Let's now convert the Jupyter notebook into a plain Python script:

In [ ]:
!jupyter nbconvert --to script cfds_lab_06.ipynb